In [1]:
%matplotlib notebook

In [10]:
# %matplotlib widget
import torch
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = f'{torch.cuda.get_device_properties(0).major}.{torch.cuda.get_device_properties(0).minor}'

from diffSPH.v2.parameters import parseDefaultParameters, parseModuleParameters
from diffSPH.v2.sampling import sampleParticles
from diffSPH.v2.util import countUniqueEntries, printState
from diffSPH.v2.plotting import updatePlot, visualizeParticles, prepVisualizationState

from diffSPH.v2.modules.integration import integrate
from diffSPH.v2.modules.neighborhood import neighborSearch
from diffSPH.v2.modules.shifting import solveShifting
from diffSPH.v2.modules.timestep import computeTimestep

from diffSPH.v2.simulationSchemes.deltaPlus import simulationStep    
from diffSPH.v2.modules.viscosity import computeViscosityParameter, setViscosityParameters
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch
import copy
import os
import datetime
from diffSPH.v2.util import computeStatistics
import tomli
from torch.profiler import profile, record_function, ProfilerActivity
from diffSPH.v2.modules.inletOutlet import processInlet, processOutlet, processForcing
from diffSPH.v2.plotting import plotRegions
from diffSPH.v2.sdf import getSDF, sdfFunctions, operatorDict
from diffSPH.v2.sampling import find_contour
from diffSPH.v2.plotUtil import plotRegionswSDF
from diffSPH.v2.sampling import sampleParticles, processBoundarySDFs, sampleNoisyParticles
from diffSPH.v2.finiteDifference import centralDifferenceStencil, continuousGradient
from diffSPH.v2.plotUtil import plotInitialParticleSampling
from diffSPH.v2.modules.neighborhood import searchNeighbors
from diffSPH.v2.plotting import visualizeParticleQuantity
from diffSPH.v2.sampling import generateRamp, sampleVelocityField
from diffSPH.v2.modules.neighborhood import searchNeighbors
from diffSPH.v2.sampling import generateRamp, getNoiseSampler
from diffSPH.v2.plotUtil import plotPotentialField
from diffSPH.v2.plotUtil import plotBoundaryParticleswGhosts
from diffSPH.v2.modules.mDBC import mDBCDensity
from diffSPH.v2.modules.boundaryViscosity import computeBoundaryVelocities
from diffSPH.v2.plotUtil import plotBoundaryVelocityFields
from diffSPH.v2.simulationSchemes.deltaPlus import simulationStep, callModule
from diffSPH.v2.modules.density import computeDensity
from diffSPH.v2.plotting import exportPlot, postProcessPlot
from diffSPH.v2.simulationSchemes.deltaPlus import checkNaN
from diffSPH.v2.modules.mDBC import buildBoundaryGhostParticles
from diffSPH.v2.modules.boundaryViscosity import computeBoundaryVelocities
from diffSPH.v2.plotUtil import plotBoundaryParticleswGhosts

configurationFile = './configurations/05_ldc.toml'

with open(configurationFile, 'r') as f:
    config = f.read()

config = tomli.loads(config)
config['compute'] = {'device': 'cuda', 'checkNaN': True}



import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--nx', type=int, default=128)
# parser.add_argument('--u_mag', type=float, default=2)
# parser.add_argument('--k', type=float, default=2)
# parser.add_argument('--TGV_override', type=bool, default=False)
# parser.add_argument('--Re', type=float, default=2000)
# parser.add_argument('--dt', type=float, default=0.001)
# parser.add_argument('--L', type=float, default=1)
# parser.add_argument('--seed', type=int, default=0)
# parser.add_argument('--baseFrequency', type=float, default=1)
# parser.add_argument('--octaves', type=int, default=4)
# parser.add_argument('--frameLimit', type=int, default=4096)
# parser.add_argument("--verbose", help="increase output verbosity",)
# parser.add_argument('--boundary', type=str, default='free-slip')
# parser.add_argument('--centerObstacle', type=bool, default=False)
# args = parser.parse_args()

# if args.boundary == 'none':
#     config['domain']['periodic'] = True
#     config['boundary']['boundary_width'] = 0
#     config['boundary']['active'] = False

# config['particle']['nx'] = args.nx
config['fluid'] = {'u_mag' : 2}
# config['diffusion']['targetRe'] = args.Re



config = parseDefaultParameters(config)
config = parseModuleParameters(config)
# config['diffusion']['alpha'] = 0.01



In [11]:
display(config)

{'timestep': {'fixed': True,
  'dt': 0.001,
  'active': True,
  'CFL': 1.5,
  'maxDt': 0.1,
  'minDt': 0.001,
  'viscosityConstraint': True,
  'accelerationConstraint': True,
  'acousticConstraint': True},
 'domain': {'adjustParticle': False,
  'adjustDomain': False,
  'dim': 2,
  'periodic': False,
  'minExtent': tensor([-0.5391, -0.5391], device='cuda:0'),
  'maxExtent': tensor([0.5391, 0.5391], device='cuda:0'),
  'periodicity': tensor([False, False], device='cuda:0')},
 'shifting': {'scheme': 'deltaSPH',
  'threshold': 0.05,
  'maxIterations': 1,
  'freeSurface': False,
  'R': 0.25,
  'n': 4,
  'CFL': 1.5,
  'computeMach': True,
  'solver': 'BiCGStab_wJacobi',
  'normalScheme': 'lambda',
  'projectionScheme': 'mat',
  'surfaceScaling': 0.1,
  'summationDensity': False,
  'useExtendedMask': False,
  'initialization': 'zero',
  'maxSolveIter': 64,
  'surfaceDetection': 'Barcasco',
  'active': True,
  'tol': 0.0001,
  'rtol': 0.0001,
  'preconditioner': 'Jacobi',
  'verbose': False,
 

In [18]:
display(config['diffusion'])

{'velocityScheme': 'deltaSPH_inviscid',
 'targetRe': -1000,
 'pi-switch': False,
 'delta': 0.1,
 'densityScheme': 'deltaSPH',
 'nu': 0.01,
 'alpha': 0.01,
 'beta': 0.0,
 'artificial-constant': 0.02,
 'eps': 1e-06,
 'nu_sph': tensor(0.0009, device='cuda:0')}